In [1]:
import napari
import numpy as np
import matplotlib.pyplot as plt
import z5py
from pathlib import Path

In [2]:
def print_key_tree(f: z5py.File):
    print(f"Key structure of z5 file {f.filename}")
    f.visititems(lambda name, obj: print(name))


def read_volume(f: z5py.File, key: str, roi: np.lib.index_tricks.IndexExpression = np.s_[:]):
    try:
        ds = f[key]
    except KeyError:
        print(f"No key {key} in file {f.filename}")
        print_key_tree(f)
        return None
        
    ds.n_threads = 8
    print(f"Reading roi {roi} of volume {key} from {f.filename}")
    vol = ds[roi]
    print(f"Read volume with shape {vol.shape}, data type {vol.dtype}")
    
    return vol

## View n5 datasets

In [3]:
scratch_dir = Path("/scratch/buglakova")
predictions_dir = scratch_dir / "F107_bin2_619-639_predictions"
predictions_dir.mkdir(parents=True, exist_ok=True)

In [4]:
n5_path = predictions_dir / "full_raw.n5"
pred_n5_path = predictions_dir / "full_predictions.n5"
multicut_n5_path = predictions_dir / "full_multicut_beta.n5"

In [5]:
f = z5py.File(pred_n5_path, "r")
f_raw = z5py.File(n5_path, "r")
f_multicut = z5py.File(multicut_n5_path, "r")

RuntimeError: [json.exception.parse_error.101] parse error at line 1, column 1: syntax error while parsing value - unexpected end of input; expected '[', '{', or a literal

In [9]:
print_key_tree(f)

Key structure of z5 file /scratch/buglakova/F107_bin2_619-639_predictions/full_predictions.n5
predictions
predictions/background
predictions/extracellular
predictions/foreground
predictions/boundaries


## Read volumes

In [6]:
roi = np.s_[235:300, :, :]

In [11]:
pred = read_volume(f, "predictions/extracellular", roi)

Reading roi (slice(235, 300, None), slice(None, None, None), slice(None, None, None)) of volume predictions/extracellular from /scratch/buglakova/F107_bin2_619-639_predictions/full_predictions.n5
Read volume with shape (65, 1506, 1647), data type float32


In [7]:
raw = read_volume(f_raw, "raw/raw_data", roi)

Reading roi (slice(235, 300, None), slice(None, None, None), slice(None, None, None)) of volume raw/raw_data from /scratch/buglakova/F107_bin2_619-639_predictions/full_raw.n5
Read volume with shape (65, 1506, 1647), data type uint8


In [8]:
ws = read_volume(f_multicut, "watershed/watershed", roi)

NameError: name 'f_multicut' is not defined

In [14]:
betas = [0.001, 0.01, 0.1, 0.5, 0.6, 0.9, 0.99, 0.999]
beta_names = ["multicut_" + str(beta) for beta in betas]
multicut = {}
for beta, name in zip(betas, beta_names):
    multicut[beta] = read_volume(f_multicut, f"segmentation/{name}", roi)


Reading roi (slice(235, 300, None), slice(None, None, None), slice(None, None, None)) of volume segmentation/multicut_0.001 from /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta.n5
Read volume with shape (65, 1506, 1647), data type uint32
Reading roi (slice(235, 300, None), slice(None, None, None), slice(None, None, None)) of volume segmentation/multicut_0.01 from /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta.n5
Read volume with shape (65, 1506, 1647), data type uint32
Reading roi (slice(235, 300, None), slice(None, None, None), slice(None, None, None)) of volume segmentation/multicut_0.1 from /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta.n5
Read volume with shape (65, 1506, 1647), data type uint32
Reading roi (slice(235, 300, None), slice(None, None, None), slice(None, None, None)) of volume segmentation/multicut_0.5 from /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta.n5
Read volume with shape (65, 150

In [15]:
boundaries = read_volume(f, "predictions/boundaries", roi)

Reading roi (slice(235, 300, None), slice(None, None, None), slice(None, None, None)) of volume predictions/boundaries from /scratch/buglakova/F107_bin2_619-639_predictions/full_predictions.n5
Read volume with shape (65, 1506, 1647), data type float32


In [16]:
binary_boundaries = (boundaries > 0.5).astype(int)

In [17]:
extra = read_volume(f, "predictions/extracellular", roi)

Reading roi (slice(235, 300, None), slice(None, None, None), slice(None, None, None)) of volume predictions/extracellular from /scratch/buglakova/F107_bin2_619-639_predictions/full_predictions.n5
Read volume with shape (65, 1506, 1647), data type float32


## Open it in napari

In [18]:
v = napari.Viewer()

In [19]:
v.add_image(raw)

<Image layer 'raw' at 0x7f2c3c1a3100>

In [20]:
v.add_image(boundaries)

<Image layer 'boundaries' at 0x7f2c347698a0>

In [19]:
v.add_image(extra)

<Image layer 'extra' at 0x7fd9983b3b50>

In [21]:
v.add_labels(ws)

<Labels layer 'ws' at 0x7f2c3466d690>

In [21]:
v.add_labels(binary_boundaries)

<Labels layer 'binary_boundaries' at 0x7fd99816af40>

In [22]:
for beta, name in zip(betas, beta_names):
    v.add_labels(multicut[beta], name=name)

In [22]:
raw.shape

(1350, 1506, 1647)